In [1]:
from mbmvpa.data.loader import BIDSDataLoader

In [2]:
from mbmvpa.models.tf_mlp import MVPA_MLP

In [3]:
!ls test_example

dataset_description.json  derivatives  sub-01


In [4]:
root = './test_example/'

In [5]:
loader = BIDSDataLoader(layout=root, 
                        feature_name='zoom2',
                        task_name='mixedgamblestask',
                        process_name='SUgamble')

/home/cheoljun/.cache/pypoetry/virtualenvs/model-based-fmri-A7ELC43w-py3.7/lib/python3.7/site-packages/bids/layout/models.py:152: FutureWarning: The 'extension' entity currently excludes the leading dot ('.'). As of version 0.14.0, it will include the leading dot. To suppress this warning and include the leading dot, use `bids.config.set_option('extension_initial_dot', True)`.
  FutureWarning)
                                                 

INFO: retrieving from BIDS Layout: ...est_example/derivatives/mbmvpa | Subjects: 1 | Sessions: 0 | Runs: 1
      task-mixedgamblestask, process-SUgamble, feature-zoom2
INFO: start loading data
INFO: loaded data info. total-2
      subject_01: X(240, 119), y(240, 1)
INFO: loaded voxel mask(46, 55, 46)
INFO: loading data done


In [6]:
X,y = loader.get_data(subject_wise=False)
voxel_mask = loader.get_voxel_mask()

In [7]:
input_shape = X.shape[1]

model = MVPA_MLP(input_shape,
                 layer_dims=[1024, 1024],
                 activation="sigmoid",
                 activation_output="linear",
                 dropout_rate=0.5,
                 val_ratio=0.05,
                 optimizer="adam",
                 loss="mse",
                 learning_rate=0.001,
                 n_epoch = 80,
                 n_patience = 15,
                 n_batch = 64,
                 n_sample = 100000,
                 use_bias = True,
                 gpu_visible_devices=[2],
                 use_bipolar_balancing = False)

In [8]:
model.reset()

In [9]:
keras_model = model.model

In [10]:
import shap

In [11]:
import numpy as np

In [12]:
# select a set of background examples to take an expectation over
background = X[np.random.choice(X.shape[0], 100, replace=False)]



In [13]:
# explain predictions of the model on four images
e = shap.DeepExplainer(keras_model, background)


Using TensorFlow backend.
keras is no longer supported, please use tf.keras instead.
Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode. See PR #1483 for discussion.


In [14]:
# ...or pass tensors directly
# e = shap.DeepExplainer((model.layers[0].input, model.layers[-1].output), background)
shap_values = e.shap_values(X[1:5])


`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.


In [18]:
X.shape

(240, 119)